In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
# Daten einlesen
df_bildungsausgaben = pd.read_excel('Bildungsausgaben_Schweiz.xlsx')
df_bip_einkommen = pd.read_excel('BIP nach Einkommensart.xlsx')
df_studierende_mint = pd.read_excel('T10 Studierende nach Studientstufe und MINT Fach seit 2014.xlsx')

In [3]:
# Spalten bereinigen und anpassen
df_bildungsausgaben.columns = [col.lower().replace(' ', '_') for col in df_bildungsausgaben.columns]
df_bip_einkommen.columns = [col.lower().replace(' ', '_') for col in df_bip_einkommen.columns]
df_studierende_mint.columns = [col.lower().replace(' ', '_') for col in df_studierende_mint.columns]

In [4]:
# Daten kombinieren
df_bildungsausgaben = df_bildungsausgaben[['jahr', 'schweiz', 'bund', 'kantone', 'gemeinden']]
df_bip_einkommen = df_bip_einkommen[['gliederung', 'arbeitnehmerentgelt', 'nettobetriebsüberschuss']]
df_studierende_mint = df_studierende_mint[['jahr', 'total', 'informatik', 'technik']]

merged_df = pd.merge(df_bildungsausgaben, df_studierende_mint, on='jahr', how='inner')
merged_df = pd.merge(merged_df, df_bip_einkommen, left_on='jahr', right_on='gliederung', how='inner')
merged_df.drop('gliederung', axis=1, inplace=True)

In [5]:
# Machine Learning Modelle
X = merged_df[['schweiz', 'bund', 'kantone', 'gemeinden', 'total', 'informatik', 'technik']]
y = merged_df['arbeitnehmerentgelt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Lineare Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_linear = linear_model.predict(X_test)

In [7]:
# Random Forest Regression
rf_model = RandomForestRegressor(random_state=42, n_estimators=100)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

In [8]:
# Performance-Metriken
metrics = {
    'Modell': ['Lineare Regression', 'Random Forest'],
    'MAE': [mean_absolute_error(y_test, y_pred_linear), mean_absolute_error(y_test, y_pred_rf)],
    'MSE': [mean_squared_error(y_test, y_pred_linear), mean_squared_error(y_test, y_pred_rf)],
    'R^2 Score': [r2_score(y_test, y_pred_linear), r2_score(y_test, y_pred_rf)]
}

In [9]:
# Ergebnisse anzeigen
results_df = pd.DataFrame(metrics)
print(results_df)

               Modell          MAE           MSE  R^2 Score
0  Lineare Regression  2916.767138  1.508768e+07   0.937380
1       Random Forest  5618.941113  4.888338e+07   0.797114


Interpretation:

Lineare Regression:

Ein sehr hoher R²-Wert von 0.94, was darauf hinweist, dass das Modell gut erklärt, wie die Bildungsmerkmale das Arbeitnehmerentgelt beeinflussen.
Der niedrigere MAE und MSE bedeuten, dass die Vorhersagen relativ genau sind.

Random Forest Regression:

Der R²-Wert von 0.80 ist niedriger als bei der linearen Regression.
Höhere Fehlerwerte (MAE und MSE) im Vergleich zur linearen Regression deuten darauf hin, dass dieses Modell in unserem spezifischen Fall weniger geeignet ist.

In [10]:
import joblib

# Speichere die Modelle
joblib.dump(linear_model, 'linear_model.pkl')
joblib.dump(rf_model, 'random_forest_model.pkl')

print("Modelle wurden erfolgreich gespeichert.")

Modelle wurden erfolgreich gespeichert.


In [12]:
import pandas as pd

# Beispiel-Testdaten mit den korrekten Spaltennamen
test_input = pd.DataFrame([[40000, 3500, 23000, 10000, 150000, 5000, 12000]], 
                          columns=['schweiz', 'bund', 'kantone', 'gemeinden', 'total', 'informatik', 'technik'])

# Vorhersage mit geladenen Modellen
prediction_linear = loaded_linear_model.predict(test_input)
prediction_rf = loaded_rf_model.predict(test_input)

print(f"Lineare Regression Vorhersage: {prediction_linear[0]}")
print(f"Random Forest Vorhersage: {prediction_rf[0]}")

Lineare Regression Vorhersage: 402545.8847947203
Random Forest Vorhersage: 403074.72429168277


### Nächste Schritte für die XAI-Analyse:
1. Modelle laden (`linear_model.pkl` und `random_forest_model.pkl`).
2. Verwende `SHAP` oder `LIME`, um die Einflussfaktoren zu erklären.
3. Beispiel:
   ```python
   import shap
   explainer = shap.Explainer(loaded_rf_model, X_train)
   shap_values = explainer(X_test)
   shap.summary_plot(shap_values, X_test)

#### 4. Projekt zusammenpacken und übergeben
- Stelle sicher, dass **alle Dateien** in deinem Projektordner enthalten sind:
  - `Python_Project_ML_Models.ipynb`
  - `linear_model.pkl`
  - `random_forest_model.pkl`
  - `requirements.txt` (damit die nächste Person alle Abhängigkeiten installieren kann)

- Packe alles in eine **ZIP-Datei** und übergib es der nächsten Person.

Das isch nur en Biespiel was Du chöntsch mache :)

